content base cosing similarity

In [1]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

In [2]:

ratings = pd.read_csv('food_ratings.csv')#opeing food_ratings with pandas
all_recipies = pd.read_csv('all_recipes.csv')#opening all recipes with pandas
recipes_and_ratings = pd.merge(ratings , all_recipies , on='recipe_id')#merging food_ratings with recipes

In [3]:
ratings.head()

,date,rating,recipe_id,review,user_id,username
0,2012-09-11,5,222234,This recipe was delicious! Instead of cherry p...,3419993,Pie84
1,2012-12-25,5,222234,Made this recipe as instructed and it was easy...,3154459,hertzen
2,2012-11-22,5,222234,This was a great recipe to make using leftover...,10370475,suziloo
3,2015-07-07,5,222234,"Tasty , simple dessert that comes out cute as ...",2304335,Chelsea M.
4,2016-12-31,5,222234,"Delicious!! I was pressed for time, so I didn'...",1967176,Amanda H.


In [4]:
all_recipies.head()

,calories,category,ingredients,ratings,recipe_id,reviews,title,total_mins
0,728,"['Desserts', 'Fruit Desserts', 'Pineapple Dess...","['2 cups all-purpose flour', '1 teaspoon salt'...",4.703704,263516,23,Chef John's Carrot Cake,95
1,692,"['Bread', 'Yeast Bread']","['1/2 cup warm milk', '1/3 cup warm water', '1...",4.666667,272625,2,Khachapuri (Georgian Cheese Bread),115
2,209,"['Soups, Stews and Chili', 'Stews', 'Beef']","['1 (1 1/2-pound) flank steak', '2 teaspoons k...",5.000000,272615,4,Ropa Vieja (Cuban Beef),690
3,505,"['Everyday Cooking', 'Vegetarian', 'Side Dishes']","['2 cups whole milk', '1 teaspoon kosher salt,...",4.333333,272461,9,"""Instant"" Mac and Cheese",28
4,291,"['Bread', 'Yeast Bread']","['1 cup warm water', '1 tablespoon white sugar...",5.000000,271049,4,Kouign-Amann,275


In [5]:
recipes_and_ratings.head()

,date,rating,recipe_id,review,user_id,username,calories,category,ingredients,ratings,reviews,title,total_mins
0,2012-09-11,5,222234,This recipe was delicious! Instead of cherry p...,3419993,Pie84,275,"['Desserts', 'Pies', 'Tarts', 'Fruit Tarts']","['6 ounces white whole wheat flour', '1 teaspo...",5.0,8,Cherry Folditup,170
1,2012-12-25,5,222234,Made this recipe as instructed and it was easy...,3154459,hertzen,275,"['Desserts', 'Pies', 'Tarts', 'Fruit Tarts']","['6 ounces white whole wheat flour', '1 teaspo...",5.0,8,Cherry Folditup,170
2,2012-11-22,5,222234,This was a great recipe to make using leftover...,10370475,suziloo,275,"['Desserts', 'Pies', 'Tarts', 'Fruit Tarts']","['6 ounces white whole wheat flour', '1 teaspo...",5.0,8,Cherry Folditup,170
3,2015-07-07,5,222234,"Tasty , simple dessert that comes out cute as ...",2304335,Chelsea M.,275,"['Desserts', 'Pies', 'Tarts', 'Fruit Tarts']","['6 ounces white whole wheat flour', '1 teaspo...",5.0,8,Cherry Folditup,170
4,2016-12-31,5,222234,"Delicious!! I was pressed for time, so I didn'...",1967176,Amanda H.,275,"['Desserts', 'Pies', 'Tarts', 'Fruit Tarts']","['6 ounces white whole wheat flour', '1 teaspo...",5.0,8,Cherry Folditup,170


In [6]:
fooddf = recipes_and_ratings.set_index('recipe_id').reset_index().drop_duplicates(subset='recipe_id')
#droping duplicate recipe_id
fooddf = fooddf.set_index('title')
fooddf.head()

,recipe_id,date,rating,review,user_id,username,calories,category,ingredients,ratings,reviews,total_mins
title,,,,,,,,,,,,
Cherry Folditup,222234,2012-09-11,5,This recipe was delicious! Instead of cherry p...,3419993,Pie84,275,"['Desserts', 'Pies', 'Tarts', 'Fruit Tarts']","['6 ounces white whole wheat flour', '1 teaspo...",5.000000,8,170
Chef John's Carrot Cake,263516,2018-06-25,2,I don't like the original recipe because it ha...,4804862,asiandelightnyc,728,"['Desserts', 'Fruit Desserts', 'Pineapple Dess...","['2 cups all-purpose flour', '1 teaspoon salt'...",4.703704,23,95
Khachapuri (Georgian Cheese Bread),272625,2019-05-13,5,Maybe my favorite bread of all time,11446901,resy,692,"['Bread', 'Yeast Bread']","['1/2 cup warm milk', '1/3 cup warm water', '1...",4.666667,2,115
Ropa Vieja (Cuban Beef),272615,2019-04-22,5,love this recipe,25039718,Jeannette Rodriguez Dejesus,209,"['Soups, Stews and Chili', 'Stews', 'Beef']","['1 (1 1/2-pound) flank steak', '2 teaspoons k...",5.000000,4,690
"""Instant"" Mac and Cheese",272461,2019-04-14,4,I prepared it with 2% milk and a light cheddar...,24992909,Jayne Doe,505,"['Everyday Cooking', 'Vegetarian', 'Side Dishes']","['2 cups whole milk', '1 teaspoon kosher salt,...",4.333333,9,28


In [11]:

tfv = TfidfVectorizer(min_df=3,max_features=None , strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}[^0-9]',ngram_range=(1,3),stop_words='english')
fooddf['ingredients'] = fooddf['ingredients'].fillna('')
tfv_matrix = tfv.fit_transform(fooddf['ingredients'])
print(tfv_matrix)

  (0, 1182)	0.12166969730535312
  (0, 5224)	0.05918836823432849
  (0, 7867)	0.13457568613728504
  (0, 7922)	0.07261026224201103
  (0, 7856)	0.12595031182484423
  (0, 3507)	0.05736262803774139
  (0, 0)	0.06807315626390165
  (0, 7194)	0.059832526809144436
  (0, 6746)	0.10330240271882264
  (0, 367)	0.07649829457147753
  (0, 480)	0.06960013458545253
  (0, 6144)	0.040792846569285036
  (0, 6943)	0.06266074398114815
  (0, 2115)	0.13085895365576414
  (0, 7699)	0.07443600243859813
  (0, 1810)	0.06743075717574111
  (0, 2984)	0.0547933143914207
  (0, 4200)	0.055767214076169844
  (0, 843)	0.0813357430087933
  (0, 4168)	0.06608810880452365
  (0, 2258)	0.09658337721306294
  (0, 997)	0.027964977545356092
  (0, 2608)	0.05301069788561688
  (0, 4143)	0.21925881379400433
  (0, 2380)	0.13246661716178507
  :	:
  (1126, 2633)	0.09088815858669616
  (1126, 7799)	0.09806911145391528
  (1126, 7831)	0.09088815858669616
  (1126, 3295)	0.07688860592618293
  (1126, 128)	0.08908860987052175
  (1126, 3297)	0.08908860

In [12]:
fooddf = fooddf.reset_index()
#sig = sigmoid_kernel(tfv_matrix,tfv_matrix)
sig = pairwise_distances(tfv_matrix,tfv_matrix,metric='cosine')
indecies = pd.Series(fooddf.index ,index=fooddf['title'])


In [13]:
id_food = indecies['Ropa Vieja (Cuban Beef)']
sig_scores = list(enumerate(sig[id_food]))
sig_scores = sorted(sig_scores , key=lambda x:x[1],reverse=True)
sig_scores

[(78, 1.0),
 (535, 1.0),
 (804, 1.0),
 (812, 1.0),
 (835, 1.0),
 (802, 0.9936022413760655),
 (635, 0.9890912140438918),
 (697, 0.9830372040683109),
 (652, 0.9818144592697912),
 (341, 0.9812391256196035),
 (143, 0.9811236266259055),
 (493, 0.9810653399554783),
 (998, 0.9806581855452503),
 (795, 0.9773116634472139),
 (791, 0.9764740468009767),
 (416, 0.97631157659174),
 (656, 0.9759815943985836),
 (625, 0.9757775270041366),
 (897, 0.9753590601284635),
 (165, 0.9720556034692099),
 (993, 0.9720261204945732),
 (1061, 0.971960102418427),
 (733, 0.9718919109719891),
 (568, 0.9717451367536876),
 (736, 0.9714067522370436),
 (849, 0.9708861030272581),
 (892, 0.9687967182077479),
 (1116, 0.9678808175607946),
 (828, 0.9677318359846983),
 (1019, 0.9673443019607167),
 (1104, 0.9663477247750826),
 (888, 0.966131475554403),
 (694, 0.9648652383836539),
 (1070, 0.9638897030522997),
 (726, 0.9635750767647114),
 (1114, 0.9635349360154176),
 (113, 0.9634970332609571),
 (544, 0.9629374713785586),
 (124, 0.9